In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.25, random_state=13)
X_train.shape, X_test.shape

((227, 13), (76, 13))

1)Обучите решающее дерево из sklearn (sklearn.tree.DecisionTreeClassifier) без ограничения на максимальную глубину (max_depth=None). В качестве сида поставьте random_state=13.

Подробнее о параметрах можно почитать в документации.

Найдите долю правильных ответов полученного алгоритма на обучающей выборке (в процентах). Ответ округлите до двух знаков после запятой. Например, если значение доли правильных ответов будет равно 0.1234, то ответом будет 12.34 - ведь это 12.34%.



Функция подсчета долю правильных ответов полученного алгоритма

In [4]:
def get_wright_answers_percent(data1, data2):
    k = 0
    for i in range(len(data1)):
        if data1[i] == data2[i]:
            k += 1
            
    return k / len(data1) * 100

Теперь обучим дерево и построим тренировочные и тестовые выборки

In [5]:
tree = DecisionTreeClassifier(random_state=13).fit(X_train, y_train)

predicted_test = tree.predict(X_test)
predicted_train = tree.predict(X_train)

Подсчитаем точность обучающей выборки

In [6]:
get_wright_answers_percent(list(predicted_train), list(y_train))

100.0

2)Теперь найдите долю правильных ответов полученного алгоритма на тестовой выборке (в процентах). Ответ округлите до двух знаков после запятой.



In [7]:
get_wright_answers_percent(list(predicted_test), list(y_test))

67.10526315789474

ВЫВОД:
Есть некоторое переобучение, но всего лишь 67%. Это не очень сильный результат

3)Подберите с помощью кросс-валидации оптимальные гиперпараметры алгоритма. Выбирайте из следующих наборов:
 - max_depth: [3, 4, 5, 6, 7, 8, 9, 10, None]
 - max_features: ['auto', 'log2', None]
 - min_samples_leaf: range(1, 10)
 - min_samples_split: range(2, 10)
 - criterion: ['gini', 'entropy']

В этом вам поможет метод sklearn.model_selection.GridSearchCV. Подробнее о том, какие параметры и методы в нем используются, и о примерах работы с ним можно прочитать в документации.


  

Какое значение глубины дерева получилось оптимальным?

1) Создайте решающее дерево - не забудьте поставить random_state=13.

In [12]:
greeded_tree = DecisionTreeClassifier(random_state=13)

2) Задайте param_grid - сетку (словарь) гиперпараметров и их значений, по которой будет проходить метод.


In [13]:
param_grid = {
    "max_depth": [3, 4, 5, 6, 7, 8, 9, 10, None],
    "max_features": ['auto', 'log2', None],
    "min_samples_leaf": range(1, 10),
    "min_samples_split": range(2, 10),
    "criterion": ['gini', 'entropy']
}

3) Вызовите метод GridSearchCV - в качестве параметра estimator задайте решающее дерево из первого шага, param_grid - сетку из второго. Задайте параметр cv=5, чтобы кросс-валидация проходила по 5 фолдам. Также задайте параметр scoring='accuracy', чтобы оценка качества моделей на кросс-валидации проходила с помощью доли правильных ответов. Запустите метод на обучающей выборке с помощью fit. 
  

In [14]:
clf = GridSearchCV(greeded_tree, param_grid, cv=5, scoring='accuracy')
clf.fit(X_test, y_test)


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=13),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, None],
                         'max_features': ['auto', 'log2', None],
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10)},
             scoring='accuracy')


  4) Выведите лучшие параметры с помощью атрибута best_params_.

In [15]:
clf.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': None,
 'min_samples_leaf': 1,
 'min_samples_split': 4}

Какое значение глубины дерева получилось оптимальным?
 - 5

Обучим дерево с лучшими параметрами

In [16]:
pred_tree = DecisionTreeClassifier(random_state=13, criterion='gini', max_depth=5, min_samples_leaf=1, min_samples_split=4).fit(X_train, y_train)
predicted_test = pred_tree.predict(X_test)
predicted_train = pred_tree.predict(X_train)

Подсчитаем точность новой обучающей выборки

In [17]:
get_wright_answers_percent(list(predicted_train), list(y_train))

94.27312775330397

Подсчитаем точность новой тестовой выборки

In [18]:
get_wright_answers_percent(list(predicted_test), list(y_test))

63.1578947368421

ВЫВОД: теперь модель не так сильно заточена под обучающую выборку, но точность тестовой выборки по прежнему маленькая,